In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

La variabile dipendente da prevedere è la temperatura percepita in base alla temperatura reale, tipo di precipitazione, velocità del vento, umidità ecc...

In [ ]:
dataset = pd.read_csv('/kaggle/input/szeged-weather/weatherHistory.csv')
y = dataset['Apparent Temperature (C)']
dataset = dataset.drop(columns = ['Formatted Date', 'Loud Cover', 'Daily Summary','Apparent Temperature (C)'])
dataset.head()


Con LabelEncoder vengono codificati i valori stringa in campi numerici

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataset['Summary'] = encoder.fit_transform(dataset['Summary'])
column = dataset['Precip Type'].apply(str)
dataset['Precip Type'] = encoder.fit_transform(column)
dataset.head()

Tramite la matrice di correlazione è possibile studiare quanto le features siano correlate tra loro. I dati non sono eccessivamente correlati tra loro, infatti, come si può vedere dai risultati, la regressione lineare ottiene un buon risultato. Vengono comunque utilizzati i metodi di regolarizzazione che mirano a ridurre la dimensione dei coefficienti stimando i pesi aggiungendo una penalità alfa.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corrmat = dataset.corr()
plt.figure(figsize=(10,10))
g = sns.heatmap(corrmat,annot=True)

Il dataset è stato diviso in set di dati di allenamento e di test. In questo caso è stato considerato il 30% del set di dati come dati di test e il restante 70% come dati di addestramento.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size = 0.3, random_state = 123)

I regolarizzatori Ridge e Lasso dei modelli lineari presuppongono che tutte le feature siano centrate attorno a 0 e abbiano stesso ordine di varianza. Se una feature presenta una varianza di ordine di grandezza più grandi di altre, potrebbe dominare la funzione obiettivo e rendere lo stimatore incapace di apprendere correttamente da altre feature come ci si aspetta. Per questo motivo trasformiamo i dati secondo la tecnica di standardizzazione secondo cui s(x) = (x - u)/sd dove u è la media dei campioni di training e sd è la deviazione standard dei campioni di training.

In [ ]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler(copy=True, with_mean=True, with_std=True)
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

Come primo metodo di regressione è stata utilizzata una rete neurale per la regressione, poi regressione lineare, ridge regression, lasso regression, elastic net regression.
Le metriche utilizzate per valutare i modelli sono:

- r2_score: restituisce il coefficiente di determinazione R^2 della previsione. Il coefficiente R^2 è definito come (1 - u / v), dove u è la somma residua dei quadrati ((y_true - y_pred) ** 2) .sum () e v è la somma totale dei quadrati ((y_true - y_true.mean ()) ** 2) .sum (). Il miglior punteggio possibile è 1.0.

- mean_squared_error: è calcolato elevando al quadrato la differenza tra il valore reale y e quello predetto. Il quadrato determina una particolarità dell’errore quadratico medio, cioè che i grandi errori sono fortemente penalizzati.

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
mlp = MLPRegressor(hidden_layer_sizes = 5, activation = 'tanh', alpha = 0.0001)
mlp.fit(X_train, y_train)
y_predtest = mlp.predict(X_test)
print('TEST SET:')
print('MSE: ', mean_squared_error(y_test, y_predtest))
print('R2: ', r2_score(y_test, y_predtest))


Regressione Lineare:

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, RidgeCV, LinearRegression, LassoCV, ElasticNetCV
lregr = LinearRegression()
lregr.fit(X_train, y_train)
y_predtest = lregr.predict(X_test)
print('TEST SET:')
print('MSE: ', mean_squared_error(y_test, y_predtest))
print('R2: ', r2_score(y_test, y_predtest))
print('Coefficienti:')
lregr.coef_

Con regressione ridge si intende un termine usato per riferirsi a un modello di regressione lineare i cui coefficienti non sono stimati dal metodo dei minimi quadrati, ma da un altro stimatore, chiamato stimatore ridge, che possiede bias ma ha una varianza inferiore rispetto ai minimi quadrati.
Lo stimatore ridge riduce i coefficienti di regressione, in modo che le variabili, con un contributo minore al risultato, abbiano i loro coefficienti vicini allo zero.

In [ ]:
#è fondamentale calcolare il valore della penalità, dato dal coefficiente alfa in sklearn.
#Per farlo sarà calcolato alfa tra questi otto valori, tramite RidgeCV, che permette di adattare
#un modello di regressione ridge con cross-validation.

alpha = [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 100, 1000]
model_cv = RidgeCV(alphas = alpha)
model_cv.fit(X_train, y_train)
print('ALPHA: ', model_cv.alpha_)

ridge_model = Ridge(alpha = 1.0)
ridge_model.fit(X_train, y_train)
y_predtest = ridge_model.predict(X_test)
print('TEST SET:')
print('MSE: ', mean_squared_error(y_test, y_predtest))
print('R2: ', r2_score(y_test, y_predtest))
print('Coefficienti:')
ridge_model.coef_

Lasso riduce i coefficienti di regressione verso lo zero penalizzando il modello di regressione con una penalità (alpha) che ha l’effetto di forzare alcune delle stime dei coefficienti, con un contributo minore al modello, a essere esattamente uguale a zero.
Ciò significa che il lasso può anche essere visto come un’alternativa ai metodi di feature selection per eseguire la selezione delle variabili al fine di ridurre la complessità del modello.

In [ ]:
#calcolo di alpha tramite LassoCV

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
alpha = [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 100, 1000]
model_cv = LassoCV(alphas = alpha)
model_cv.fit(X_train, y_train)
print('ALPHA: ', model_cv.alpha_)

lasso_model = Lasso(alpha = 0.0001)
lasso_model.fit(X_train, y_train)
y_predtest = lasso_model.predict(X_test)
print('TEST SET:')
print('MSE: ', mean_squared_error(y_test, y_predtest))
print('R2: ', r2_score(y_test, y_predtest))
print('Coefficienti:')
lasso_model.coef_

Elastic Net combina le proprietà della regressione di Ridge e Lasso.

In [ ]:
#calcolo di alpha tramite ElasticNetCV

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
alpha = [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 100, 1000]
model_cv = ElasticNetCV(alphas = alpha)
model_cv.fit(X_train, y_train)
print('ALPHA: ', model_cv.alpha_)

elastic_model = ElasticNet(alpha = 0.0001)
elastic_model.fit(X_train, y_train)
y_predtest = elastic_model.predict(X_test)
print('TEST SET:')
print('MSE: ', mean_squared_error(y_test, y_predtest))
print('R2: ', r2_score(y_test, y_predtest))
print('Coefficienti:')
elastic_model.coef_